In [0]:
import os, sys, tempfile, shutil
import multiprocessing as mp
from IPython.parallel import Client
import pandas as pd
import time
from IPython.display import clear_output, FileLink

In [0]:
rc = Client(profile="huge")
dview = rc[:]
lview = rc.load_balanced_view()
len(rc)

In [0]:
def get_idle_engines(rc):
    qs = rc.queue_status()
    time.sleep(10)
    active = [eid for eid in sorted(qs)[:-1] if not qs[eid]['queue']]
    d = rc[active]
    l = rc.load_balanced_view(targets=active)
    return d, l

In [0]:
dview, lview = get_idle_engines(rc)
len(dview)

In [0]:
with dview.sync_imports():
    import os, stopwatch, multiprocessing, tempfile, shutil, socket

In [0]:
def get_single_host_lview(host_list):
    hosts = dview.apply(socket.gethostname).get()
    host_ids = {}
    for i, host in enumerate(hosts):
        if not host in host_ids:
            host_ids[host] = []
        if host_list == "all":
            host_ids[host].append(i)
        elif host in host_list:
            host_ids[host].append(i)
    hview = [v[0] for k, v in host_ids.items()]
    hlview = rc.load_balanced_view(targets=hview)
    return hlview

In [0]:
hlview = get_single_host_lview("all")
len(hlview)

In [0]:
assembly = "/data7/cfriedline/assemblies/foxtail2/Green_26_ATCGCGCAA.fastq_31_data_31/contigs.fa_in_map.fa"

In [0]:
fastq_dir = "/data7/eckertlab/projects/ethan/HiSeq_140603/FASTQ"

In [0]:
fastq_files = !ls $fastq_dir | grep 'fastq$' | grep -v processed
fastq_files = sorted([os.path.join(fastq_dir, x) for x in fastq_files])

In [0]:
analysis_dir = "/data7/eckertlab/projects/ethan/analysis/gatk"
picard = "/home/cfriedline/data7/src/broadinstitute-picard-03a1d72/dist/picard.jar"
java = "/home/cfriedline/jdk1.7.0_25/bin/java"
bowtie2_dir = "/home/cfriedline/data7/src/bowtie2-2.2.4"
bowtie2_build = os.path.join(bowtie2_dir, "bowtie2-build")
bowtie2 = os.path.join(bowtie2_dir, "bowtie2")

In [0]:
def create_library_index(barcode_files):
    index = {}
    for f in barcode_files:
        df = pd.read_csv(f, sep="\t")
        names = df.apply(lambda row: "%s_%s" % (row.Family_ID, row.Sample_ID), axis=1)
        for n in names:
            if not n in index:
                index[n] = os.path.basename(f).replace(".csv", "")
            else:
                print "duplicate sample found at %s ()" % (n, os.path.basename(f))
    return index
        
library_index = create_library_index(["/data7/eckertlab/projects/ethan/ethan_library1.csv",
                                      "/data7/eckertlab/projects/ethan/ethan_library2.csv"])

In [0]:
def get_sample_name_from_file(f):
    base = os.path.basename(f)
    sample_name = "_".join(base.split("_")[0:2])
    return sample_name

In [0]:
def get_async_progress(jobs):
    ready = 0
    for j in jobs:
        if j.ready():
            ready += 1
    return "%d/%d" % (ready, len(jobs))

In [0]:
x = !$bowtie2_build -f $assembly $assembly

In [0]:
#modified from read_mapping notebook to align with LB and PL in @RG
def run_bowtie2(args):
    timer = stopwatch.Timer()
    cpus = multiprocessing.cpu_count()
    bowtie2, assembly, reads, rglb, sam = args
    tmp = tempfile.NamedTemporaryFile(delete=False)
    rgid = os.path.basename(reads)
    rgsm = rgid
    cmd = "%s %s %s -p %d -x %s -U %s -S \
    %s --rg-id %s --rg SM:%s --rg PL:illumina --rg LB:%s" % (bowtie2,
                                                           "--local",
                                                           "--very-sensitive-local", 
                                                           cpus, 
                                                           assembly,
                                                           reads,
                                                           tmp.name,
                                                           rgid,
                                                           rgsm,
                                                           rglb)
    print socket.gethostname(), cmd
    !$cmd
    shutil.move(tmp.name, sam)
    timer.stop()
    return assembly, sam, cmd, timer.elapsed
dview['run_bowtie2'] = run_bowtie2

In [0]:
bowtie2_jobs = []
for f in fastq_files:
    sample_name = get_sample_name_from_file(f)
    rglb = library_index[sample_name]
    sam = os.path.join(analysis_dir, "%s_bowtie2.sam" % os.path.basename(f))
    bowtie2_jobs.append(hlview.apply_async(run_bowtie2, (bowtie2, assembly, f, rglb, sam)))

In [0]:
print get_async_progress(bowtie2_jobs)

In [0]:
def run_sortsam(args):
    java, picard, sam_file = args
    out_bam = "%s_sorted.bam" % sam_file
    t = tempfile.NamedTemporaryFile(delete=False)
    cmd = "%s -jar %s SortSam INPUT=%s OUTPUT=%s SORT_ORDER=coordinate" % (java,
                                                                               picard,
                                                                               sam_file,
                                                                               t.name)
    print cmd
    !$cmd
    shutil.move(t.name, out_bam)
    return cmd, out_bam
dview['run_sortsam'] = run_sortsam

In [0]:
sam_files = !ls {analysis_dir}/*_bowtie2.sam
sam_files = sorted(sam_files)

In [0]:
run_sortsam((java, picard, sam_files[0]))

In [0]:
sortsam_jobs = []
for f in sam_files:
    sortsam_jobs.append(lview.apply_async(run_sortsam, (java, picard, f)))

In [0]:
get_async_progress(sortsam_jobs)

In [0]:
def mark_duplicates(args):
    java, picard, bam_file = args
    out_bam = "%s_dedup.bam" % bam_file
    t = tempfile.NamedTemporaryFile(delete=False)
    cmd = "%s -jar %s MarkDuplicates \
    INPUT=%s OUTPUT=%s METRICS_FILE=%s.metrics" %     (java,
                              picard,
                              bam_file,
                              t.name,
                              out_bam)
    print cmd
    !$cmd
    shutil.move(t.name, out_bam)
    return cmd, out_bam
dview['mark_duplicates'] = mark_duplicates

In [0]:
sorted_bams = !ls {analysis_dir}/*bowtie2*_sorted.bam
sorted_bams = sorted(sorted_bams)

In [0]:
dedup_jobs = []
for f in sorted_bams:
    dedup_jobs.append(hlview.apply_async(mark_duplicates, (java, picard, f)))

In [0]:
get_async_progress(dedup_jobs)

In [0]:
def build_bam_index(args):
    java, picard, bam_file = args
    cmd = "%s -jar %s BuildBamIndex INPUT=%s" % (java, picard, bam_file)
    print cmd
    !$cmd
    return cmd
dview['build_bam_index'] = build_bam_index

In [0]:
dedup_bams = !ls {analysis_dir}/*bowtie2.sam_sorted.bam_dedup.bam
dedup_bams = sorted(dedup_files)

In [0]:
index_jobs = []
for f in dedup_bams:
    index_jobs.append(hlview.apply_async(build_bam_index, (java, picard, f)))

In [0]:
get_async_progress(index_jobs)

In [0]:
gatk = "/home/cfriedline/data7/src/GATK3.3/GenomeAnalysisTK.jar"

In [0]:
!$java -jar $picard CreateSequenceDictionary REFERENCE=$assembly output={assembly}.dict

In [0]:
def run_realigner_target_creator(args):
    print socket.gethostname()
    java, gatk, assembly, bam_file_list, out_dir = args
    out_file = os.path.join(out_dir, "forIndelRealigner.intervals")
    cmd = "%s -Xmx10G -jar %s -T RealignerTargetCreator -R %s -I %s -o %s" % (java,
                                                                              gatk,
                                                                              assembly,
                                                                              bam_file_list,
                                                                              out_file)
    print cmd
    !$cmd
    return cmd
dview['run_realigner_target_creator'] = run_realigner_target_creator

In [0]:
with open(os.path.join(analysis_dir, "dedup_bams.list"), "w") as o:
    for f in dedup_bams:
        o.write("%s\n" % f)

In [0]:
realiger_target_creator_job = lview.apply_async(run_realigner_target_creator, (java, 
                              gatk, 
                              assembly, 
                              "/data7/eckertlab/projects/ethan/analysis/gatk/dedup_bams.list",
                              analysis_dir))

In [0]:
print realiger_target_creator_job.stdout[-150:]

In [0]:
def run_indel_realigner(args):
    print socket.gethostname()
    print args
    java, gatk, assembly, bam_file_list, out_dir, intervals = args
    cmd = "%s -Xmx100G -jar %s -T IndelRealigner -R %s -I %s -nWayOut '%s' -targetIntervals %s" % (java,
                                                                                          gatk,
                                                                                          assembly,
                                                                                          bam_file_list,
                                                                                          "_cleaned.bam",
                                                                                          intervals)
    print cmd
    #!$cmd
    return cmd
dview['run_indel_realigner'] = run_indel_realigner

In [0]:
indel_realiger_job = lview.apply_async(run_indel_realigner, (java,
                                                             gatk,
                                                             assembly,
                                                             "/data7/eckertlab/projects/ethan/analysis/gatk/dedup_bams.list",
                                                             analysis_dir,
                                                             "/data7/eckertlab/projects/ethan/analysis/gatk/forIndelRealigner.intervals"))

In [0]:
#this command run manually on godel in tmux
print indel_realiger_job.get()

In [0]:
realigned_bams = !ls {analysis_dir}/*dedup_realigned.bam
realigned_bams = sorted(realigned_bams)

##Setup base recalibration

This sets up the framework for BSQR, but will be run on uncalibrated SNPs first 
per GATK best practices.  e.g., 

* First do an initial round of SNP calling on your original, unrecalibrated data.
* Then take the SNPs that you have the highest confidence in and use that set as the database of known SNPs by feeding it as a VCF file to the base quality score recalibrator.
* Finally, do a real round of SNP calling with the recalibrated data. These steps could be repeated several times until convergence.

In [0]:

def run_base_recalibrator(args):
    java, gatk, assembly, bam_file, out_dir = args
    cmd_args = (java, gatk, assembly, bam_file, bam_file, bam_file)
    
    cmd1 = "%s -jar %s \
    -T BaseRecalibrator \
    -R %s \
    -I %s \
    -o %s_recal_data.table" % cmd_args[:-1] 
    
    cmd2 = "%s -jar %s \
    -T BaseRecalibrator \
    -R %s \
    -I %s \
    -BQSR %s_recal_data.table \
    -o %s_post_recal_data.table" % cmd_args
    
    cmd3 = "%s -jar %s \
    -T AnalyzeCovariates \
    -R %s \
    -before %s_recal_data.table \
    -after %s_post_recal_data.table \
    -plots %s_recalibration_plots.pdf" % cmd_args
    
    cmd4 = "%s -jar %s \
    -T PrintReads \
    -R %s \
    -I %s \
    -BQSR %s_recal_data.table \
    -o %s_bsqr.bam" % cmd_args
    
    print socket.gethostname()
    
    for cmd in [cmd1, cmd2, cmd3, cmd4]:
        print cmd
        !$cmd
        
    return [cmd1, cmd2, cmd3, cmd4]
dview['run_base_recalibrator'] = run_base_recalibrator

In [0]:
#for f in realigned_bams:
#     bqsr_jobs.append(hlview.apply_async(run_base_recalibrator, (java, 
#                                                                 gatk, 
#                                                                 assembly, 
#                                                                 realigned_bams[0], 
#                                                                 analysis_dir)))

##Initial variant calling

In [0]:
def run_haploptype_caller(args):
    java, gatk, assembly, bam_file = args
    cmd = "%s -jar %s \
     -T HaplotypeCaller \
     -R %s \
     -I %s \
     --emitRefConfidence GVCF \
     --variant_index_type LINEAR \
     --variant_index_parameter 128000 \
     -o %s.raw.snps.indels.g.vcf" % (java, 
                                     gatk,
                                     assembly,
                                     bam_file,
                                     bam_file)
    print socket.gethostname()
    !$cmd
    return cmd
dview['run_haploptype_caller'] = run_haploptype_caller

In [0]:
var_jobs = []
for f in realigned_bams:
    var_jobs.append(hlview.apply_async(run_haploptype_caller, (java,
                                                               gatk,
                                                               assembly,
                                                               f)))

In [0]:
get_async_progress(var_jobs)

In [0]:
gvcf_files = !ls {analysis_dir}/*.g.vcf
len(gvcf_files)

In [0]:
def run_genotype_gvcfs(args):
    java, gatk, vcf_files, output_dir = args
    out_vcf = os.path.join(output_dir, "foxtail_gatk.vcf")
    variant_string = " --variant ".join(vcf_files)
    cmd = "%s -Xmx100g -jar %s \
       -R %s \
       -T GenotypeGVCFs \
       --variant %s \
       -o %s" % (java, 
                 gatk,
                 assembly,
                 variant_string,
                 out_vcf)
    print socket.gethostname()
#     !$cmd
    return cmd
dview['run_genotype_gvcfs'] = run_genotype_gvcfs

In [0]:
with open(os.path.join(analysis_dir, "run_gvcf.sh"), "w") as o:
    cmd = run_genotype_gvcfs((java, gatk, gvcf_files, analysis_dir))
    o.write("%s\n" % cmd)